In [1]:
from datasets import load_dataset

# Load all languages with dev and devtest splits
ds_full = load_dataset("openlanguagedata/flores_plus")

# Load only the dev split for all languages
ds_dev = load_dataset("openlanguagedata/flores_plus", split="dev")



/home/snt/projects_lujun/mt_reasoning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating devtest split: 100%|██████████| 216568/216568 [00:01<00:00, 122531.42 examples/s]


In [8]:
import pandas as pd

all_data = []
for split in ds_full.keys():
    all_data.extend(ds_full[split])

df_full = pd.DataFrame(all_data)

In [12]:
df_full.to_json("data/eval/flore200.jsonl",lines=True, orient="records")

In [ ]:
# uv run vllm serve /home/snt/projects_lujun/base_models/Qwen3-4B-Thinking-2507 --gpu_memory_utilization 0.5 --max_model_len 32768 --port 8000 --host 0.0.0.0

# uv run vllm serve /home/snt/projects_lujun/base_models/DeepSeek-R1-Distill-Llama-8B --enable-reasoning --reasoning-parser deepseek_r1 --gpu_memory_utilization 0.5 --max_model_len 32768 --port 8000 --host 0.0.0.0 

import pandas as pd
import requests
from openai import OpenAI
import importlib
from datetime import datetime
from mt_reason.util import prompts_util, clients_util

importlib.reload(prompts_util)
importlib.reload(clients_util)

import os
ts = datetime.now().strftime("%m%d%H%M")  

#### Setttings

df = pd.read_json("data/source/flore200.jsonl", lines=True)
openai_api_key = "sk-pro"
openai_api_base = "http://0.0.0.0:8000/v1/chat/completions"
openai_api_base = "https://api.openai.com/v1/chat/completions"
# model_path = "/home/snt/projects_lujun/base_models/Qwen3-4B-Thinking-2507"
model_path = "/home/snt/projects_lujun/base_models/DeepSeek-R1-Distill-Llama-8B"
# model_path = "o3-mini-2025-01-31"
output_path_folder = "data/outputs/flore200_eval"
iso639_df = prompts_util.load_iso639_df()
tgt_langs = [ "Assamese", "Luxembourgish", "Maltese", "Javanese", "Lingala", "Hindi", "German", "Modern Standard Arabic", "Standard Malay","Swahili"]
tgt_codes = [ "asm", "ltz", "mlt", "jav", "lin", "hin", "ger", "ara", "may", "swa"]


client = OpenAI(api_key=openai_api_key, base_url=openai_api_base)
eng_code = "eng"
all_langs_code = df['iso_639_3'].dropna().unique()
df_eng = df[df['iso_639_3'] == eng_code]

for tgt_code in tgt_codes:
    ### English To LRLs
    out_path = f"{output_path_folder}/results_{eng_code}-{tgt_code}_{ts}.jsonl"
    if os.path.exists(out_path):
        try:
            df_prev = pd.read_json(out_path, lines=True)
            idx_prev = len(df_prev)
        except Exception:
            idx_prev = 0
    else:
        idx_prev = 0

    for idx, row in df_eng.iterrows():
        if idx < idx_prev:
            continue

        update_row = row.copy()
        translation_prompt = prompts_util.translate_prompt(row['text'], eng_code, tgt_code)
        reasoning_started_at = datetime.now()
        translation, translation_reasoning_path = clients_util.generate_with_calling_vllm(openai_api_base, model_path, translation_prompt, api_key=openai_api_key)
        reasoning_ended_at = datetime.now()
        reasoning_elapsed_sec = (reasoning_ended_at - reasoning_started_at).total_seconds()
        update_row['translation'] = translation
        update_row['translation_reasoning_path'] = str(translation_reasoning_path)
        update_row['reasoning_started_at'] = reasoning_started_at.isoformat()
        update_row['reasoning_ended_at'] = reasoning_ended_at.isoformat()
        update_row['reasoning_elapsed_sec'] = reasoning_elapsed_sec
        updated_df = pd.DataFrame([update_row])
        mode = "w" if idx_prev == 0 else "a"
        updated_df.to_json(out_path, orient="records", lines=True, mode=mode)

    ### LRLs To English
    out_path = f"{output_path_folder}/results_{tgt_code}-{eng_code}.jsonl"
    if os.path.exists(out_path):
        try:
            df_prev = pd.read_json(out_path, lines=True)
            idx_prev = len(df_prev)
        except Exception:
            idx_prev = 0
    else:
        idx_prev = 0

    for idx, row in df_eng.iterrows():
        if idx < idx_prev:
            continue

        update_row = row.copy()
        translation_prompt = prompts_util.translate_prompt(row['text'], tgt_code, eng_code)
        translation = clients_util.generate_with_calling_vllm(openai_api_base, model_path, translation_prompt)
        update_row['translation'] = translation
        updated_df = pd.DataFrame([update_row])
        mode = "w" if idx_prev == 0 else "a"
        updated_df.to_json(out_path, orient="records", lines=True, mode=mode)

In [ ]:
uv run vllm serve /home/snt/projects_lujun/base_models/Qwen3-4B-Thinking-2507 --gpu_memory_utilization 0.5 --max_model_len 32768 --port 8000 --host 0.0.0.0

uv run vllm serve /home/snt/projects_lujun/base_models/DeepSeek-R1-Distill-Llama-8B --enable-reasoning --reasoning-parser deepseek_r1 --gpu_memory_utilization 0.5 --max_model_len 32768 --port 8000 --host 0.0.0.0 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/home/snt/projects_lujun/base_models/Qwen3-4B-Thinking-2507"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content) # no opening <think> tag
print("content:", content)


/home/snt/projects_lujun/mt_reasoning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.08it/s]


thinking content: Okay, the user asked for a short introduction to large language models. Hmm, they probably want something concise but clear—maybe they're a beginner or just need a quick refresher. I should avoid jargon overload since "short" is key here.  

First, what's the absolute core? LLMs are AI systems that predict human-like text by learning from massive data. Gotta emphasize "large" (billions of parameters!) and "language" (text generation, understanding). User might confuse them with chatbots, so I should clarify they're not just for chatting—they do reasoning, coding, etc.  

Wait, they said "short," so I'll skip technical details like transformer architecture. But should I mention training data? Yes, briefly—it's why they know facts but have biases. Also, real-world use cases (writing, coding) make it relatable.  

*Potential pitfalls*:  
- Don't dive into fine-tuning or prompt engineering (too long).  
- Avoid calling them "robots" (misleading).  
- Clarify they're stati